# Feature extraction (RAVDESS-dataset)

In [1]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm

In [53]:
def process_files(src_folder, target_folder):
    x_data = os.path.join(target_folder,"x_data")
    
    if not os.path.exists(x_data):
        os.mkdir(x_data)
    
    subfolders = [f.path for f in os.scandir(src_folder) if f.is_dir()]
    i = 0
    
    for subfolder in tqdm(subfolders): 
        files = os.listdir(subfolder)
        sub_labels = []
        sub_actors = []
        j = 0
        
        # for _, _, files in os.walk(src_folder):
        for file in files:
            if file.endswith(".wav"):

                # Make the five spectral features, join in tensor and save tensor as file
                y, sr = librosa.load(subfolder + '/' + file)

                #Spectrogram
                S = np.abs(librosa.stft(y))
                Y = S**2

                # mel-scaled spectrogram
                mel_spectrogram = librosa.feature.melspectrogram(S=Y, sr=sr)
                # print("Mel-spectrogram:", mel_spectrogram.shape)

                # MFCC
                mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spectrogram))
                # print("MFCC:", mfcc.shape)

                # Chromagram (using STFT)
                chromagram = librosa.feature.chroma_stft(S=S, sr=sr)
                # print("Chromagram:", chromagram.shape)

                # Spectral contrast feature
                contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
                # print("Contrast:", contrast.shape)

                # Tonnetz representation
                y = librosa.effects.harmonic(y)
                tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
                # print("Tonnetz:", tonnetz.shape)

                # Concatenate spectral features
                concat_features = np.concatenate((mel_spectrogram, mfcc, chromagram, contrast, tonnetz), axis=0)
                # print("CONCAT:", concat_features.shape, "\n")

                sub_labels.append(int(file[7]))
                sub_actors.append(int(file[-6:-4]))
                x_file = "x"+str(i)
                np.save(os.path.join(x_data, x_file), concat_features)
                i += 1
                j += 1
                
        if i == j:
            labels = sub_labels
            actors = sub_actors
        else:
            labels = labels + sub_labels
            actors = actors + sub_actors
            
    np.save(os.path.join(target_folder, "y"), np.array(labels, dtype=np.int8))
    np.save(os.path.join(target_folder, "actors"), np.array(actors, dtype=np.int8))

## Processing speech

In [54]:
src_folder = 'dataset/Audio_Speech_Actors_01-24/'
target_folder = 'processed_data/speech/'

In [55]:
process_files(src_folder, target_folder)

100%|███████████████████████████████████████████| 24/24 [07:52<00:00, 19.67s/it]


In [56]:
y = np.load(target_folder + 'y.npy')
folder = target_folder + 'x_data'
xes = [file for file in os.listdir(folder) if file.endswith(".npy")]
actors = np.load(target_folder + 'actors.npy')

print(y.shape)
print(actors.shape)
print(len(xes))

(1056,)
(1056,)
1056


## Processing song

In [60]:
src_folder = 'dataset/Audio_Song_Actors_01-24'
target_folder = 'processed_data/song/'

In [58]:
process_files(src_folder, target_folder)

100%|███████████████████████████████████████████| 24/24 [10:20<00:00, 25.84s/it]


In [61]:
y = np.load(target_folder + 'y.npy')
folder = target_folder + 'x_data'
xes = [file for file in os.listdir(folder) if file.endswith(".npy")]
actors = np.load(target_folder + 'actors.npy')

print(y.shape)
print(actors.shape)
print(len(xes))

(1012,)
(1012,)
1012
